In [1]:
!pip install tabula-py 
import requests
import pandas as pd
import time
import tabula
from google.colab import data_table
import re
import math

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.0 MB 4.8 MB/s 


In [2]:
!pip install fake_useragent

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 50 kB 2.8 MB/s 


In [3]:
from fake_useragent import UserAgent
UserAgent().chrome

'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/534.30 (KHTML, like Gecko) Chrome/12.0.742.53 Safari/534.30'

In [61]:
df=pd.DataFrame()
dffull=pd.DataFrame()

i=0
offset_ = 0
limit_ = 100
user_agent = {'User-agent': 'Mozilla/5.0'}
#response  = requests.get(url, headers = user_agent ) #, config=debug)
while i<4:
#  url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/kn/object?offset={offset_}&limit={limit_}&sortField=devId.devShortCleanNm&sortType=asc&objStatus=0'
  url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/kn/object?objStatus=0&offset={offset_}&limit=100&place=63&residentialBuildings=1'
  res = requests.get(url, headers={'User-Agent': UserAgent().chrome}) #,params=params)
  print(str(res)+''+str(offset_))
  if str(res)!='<Response [200]>': break
  offset_ = offset_ + 99
  objects_data = res.json()
  objects_list = objects_data.get('data')#.get('list') 
  df = pd.json_normalize(objects_list)
  dffull=pd.concat([dffull,df],ignore_index=True)
  time.sleep(6)
  i=i+1

<Response [200]>0
<Response [200]>99
<Response [200]>198
<Response [200]>297


In [62]:
dffull['list']

0    [{'hobjId': 18187, 'objId': 3402, 'developer':...
1    [{'hobjId': 124755, 'objId': 44163, 'developer...
2                                                   []
3                                                   []
Name: list, dtype: object

In [63]:
dffull.reset_index(inplace=True)
ser= dffull['list']
zastr=pd.DataFrame(ser[0])
i=0
while i<len(ser):
  zastr=pd.concat([zastr,pd.DataFrame(ser[i])],ignore_index=True)
  i=i+1
zastr=zastr[zastr['rpdRegionCd']==63]
zastr.reset_index(inplace=True)

In [64]:
len(zastr.objId.unique())

173

In [12]:
documents = []
i=0
dffull=pd.DataFrame()
df=dffull
while i<len(zastr.objId.unique()):
 url_doc_list = 'https://наш.дом.рф/сервисы/api/object/'+str(zastr.objId.unique()[i])
 response = requests.get(url_doc_list, headers={'User-Agent': UserAgent().chrome})
 documents = response.json()
 documents = documents['data']
 df = pd.json_normalize(documents)
 dffull=pd.concat([dffull,df],ignore_index=True)
 time.sleep(5)
 i=i+1

In [21]:
dffull.head()

,index,list,total
0,0,"[{'hobjId': 18187, 'objId': 3402, 'developer':...",173
1,1,"[{'hobjId': 18187, 'objId': 3402, 'developer':...",173
2,2,"[{'hobjId': 18187, 'objId': 3402, 'developer':...",173


In [24]:
dffull.list[1]

[{'hobjId': 18187,
  'objId': 3402,
  'developer': {'devId': 1201,
   'shortName': 'КОНСТРУКТОРСКОЕ БЮРО ПРОЕКТ: КУЙБЫШЕВ',
   'fullName': 'КОНСТРУКТОРСКОЕ БЮРО ПРОЕКТ: КУЙБЫШЕВ',
   'groupName': 'Строительный холдинг Град',
   'orgForm': {'id': 4,
    'fullForm': 'Общество с ограниченной ответственностью',
    'shortForm': 'ООО'},
   'companyGroup': 5847,
   'devInn': '6315611205'},
  'rpdRegionCd': 63,
  'objAddr': 'г Самара, район Кировский, улица Демократическая, д. 3',
  'objCommercNm': 'ЖК "КУЙБЫШЕВ" ДОМ 3',
  'objFloorMin': 5,
  'objFloorMax': 5,
  'objElemLivingCnt': 24,
  'objReady100PercDt': '2022-06-30',
  'objSquareLiving': 1096.6,
  'objCompensFundFlg': 1,
  'objProblemFlg': 0,
  'latitude': 53.2865,
  'longitude': 50.2226,
  'complexShortName': 'Куйбышев',
  'problemFlag': 'NONE',
  'siteStatus': 'Строящиеся',
  'objGuarantyEscrowFlg': 0,
  'residentialBuildings': 1,
  'rpdNum': '63-000022',
  'pdId': 2537,
  'buildType': 'Жилое',
  'objStatus': 0},
 {'hobjId': 18200,
  '

In [ ]:
x=0
i=0
documents=[]
dfrasp=pd.DataFrame()
dfraspfull=pd.DataFrame()
while i<len(dffull):
 url_doc_list = 'https://наш.дом.рф/сервисы/api/object/'+str(dffull['id'].unique()[i])+'/sales_agg'
 response = requests.get(url_doc_list, headers={'User-Agent': UserAgent().chrome})
 documents = response.json()
 documents = documents['data']
 dfrasp = pd.json_normalize(documents)
 try:
   dffull.loc[i,'raspr']=dfrasp['apartmentsAggData.realised'][0]
 except KeyError:
   qq=True
# dfraspfull=pd.concat([dfraspfull,dfrasp],ignore_index=True)
 time.sleep(5)
 i=i+1

In [ ]:
dd=dffull

In [ ]:
dd[dd['objElemLivingCnt']==0]

,id,hobjId,pdId,region,address,floorMin,floorMax,objElemLivingCnt,objReady100PercDt,wallMaterialShortDesc,...,metro.id,metro.name,metro.line,metro.color,metro.time,metro.isWalk,metro.colors,nameObj,complexShortNm,raspr
8,24540,19966,69105,63,"г Самара, Белорусская",1,1,0,2022-12-31,Другое,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"ЖК ""Две реки""",NaN,NaN
18,27564,24976,97303,63,"г Самара, район Ленинский, улица Галактионовск...",9,9,0,2023-03-31,Монолит-кирпич,...,54.509,Алабинская,Первая,#CD0505,15.516667,True,[#CD0505],"ЖК ""Гранд Империалъ""",NaN,0.0
26,24083,41346,62935,63,"Самара, улица Красных Коммунаров/Двадцать втор...",3,4,0,2023-06-30,Монолит-кирпич,...,54.313,Безымянка,Первая,#CD0505,11.933333,True,[#CD0505],"ФОК со вспомогательной инфраструктурой ""Рекорд""",NaN,0.0
37,47035,127666,535315,63,"г Самара, Пятая просека, строение Гараж-стоянка",6,7,0,2024-09-30,Монолит-кирпич,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ЖК Радужный Омега,NaN,NaN
42,40735,121247,467911,63,"г Самара, улица Санфировой, д. 95В",9,9,0,2023-06-30,Монолит-кирпич,...,54.309,Гагаринская,Первая,#CD0505,21.383333,False,[#CD0505],"ЖК ""КАПИТАЛ""",NaN,0.0
72,39646,120080,447927,63,"г Самара, ул Георгия Димитрова, блок-секция Тр...",1,1,0,2023-09-30,Другое,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ЖК Приволжский,NaN,NaN
74,39645,120081,447927,63,"г Самара, ул Георгия Димитрова, блок-секция Па...",2,2,0,2023-09-30,Монолит-кирпич,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ЖК Приволжский,NaN,0.0
78,43062,123637,512412,63,"г Самара, ул Советской Армии",1,1,0,2023-12-31,Другое,...,54.308,Московская,Первая,#CD0505,20.983333,False,[#CD0505],"""Олимп"" Жилой комплекс",NaN,NaN
80,43678,124261,506436,63,"г Самара, границы улиц, блок-секция ПАРКИНГ",1,1,0,2024-09-30,Монолит-кирпич,...,54.309,Гагаринская,Первая,#CD0505,9.366667,True,[#CD0505],"Жилой комплекс ""СОКОЛ""",NaN,NaN
89,38153,118556,406303,63,"г Самара, район Кировский, ш Ракитовское",1,1,0,2024-09-30,Другое,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
dd=dd.drop(dd[dd['objElemLivingCnt']==0].index)

In [ ]:
dfull=dd

In [ ]:
dffull.reset_index(inplace=True)

In [ ]:
dffull.drop('index',axis=1,inplace=True)

In [ ]:
dffull.head(10)

,id,hobjId,pdId,region,address,floorMin,floorMax,objElemLivingCnt,objReady100PercDt,wallMaterialShortDesc,...,metro.id,metro.name,metro.line,metro.color,metro.time,metro.isWalk,metro.colors,nameObj,complexShortNm,raspr
0,18136,20249,43084,63,"г Самара, улица Советской Армии, д. 0, корпус ...",27,27,360,2023-06-30,Монолит-кирпич,...,54.311,Советская,Первая,#CD0505,11.50,True,[#CD0505],NaN,NaN,176.0
1,4024,23666,2200,63,"г Самара, район Октябрьский, просека Пятая",5,5,16,2023-12-31,Кирпич,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Жилой комплекс ""Акварель"" (ЖК ""Акварель"") дом 9",Акварель,3.0
2,4025,23664,2200,63,"г Самара, район Октябрьский, просека Пятая",5,5,64,2022-12-31,Кирпич,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Жилой комплекс ""Акварель"" (ЖК ""Акварель"") дом 7",Акварель,64.0
3,4023,23665,2200,63,"г Самара, район Октябрьский, просека Пятая",5,5,48,2022-12-31,Кирпич,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Жилой комплекс ""Акварель"" (ЖК ""Акварель"") дом 8",Акварель,48.0
4,4026,23667,2200,63,"г Самара, район Октябрьский, просека Пятая",5,5,64,2022-12-31,Кирпич,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Жилой комплекс ""Акварель"" (ЖК ""Акварель"") дом 10",Акварель,64.0
5,47153,127787,615536,63,"г Чапаевск, ул Пролетарская, д. 10",5,5,32,2023-09-30,Кирпич,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26.0
6,47152,127788,615536,63,"г Чапаевск, ул Пролетарская, д. 10",5,5,32,2023-09-30,Кирпич,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,24539,19971,69105,63,"г Самара, Белорусская",25,25,396,2022-12-31,Кирпич,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"ЖК ""Две реки""",NaN,170.0
8,24540,19966,69105,63,"г Самара, Белорусская",1,1,0,2022-12-31,Другое,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"ЖК ""Две реки""",NaN,NaN
9,35909,116056,279525,63,"г Самара, Ленинский район, ул Мичурина, д. Б/н",23,23,101,2022-12-31,Монолит-кирпич,...,54.509,Алабинская,Первая,#CD0505,16.25,True,[#CD0505],"ЖК ""АКВАРИУМ""",NaN,66.0


In [ ]:
x=0
dffull.fillna(0)
while x<len(dffull):
  rpdPdfLink=dffull.rpdPdfLink[x]
  new=False
  print(rpdPdfLink)
  print(str(x)+':'+str(dffull[(dffull.rpdPdfLink==rpdPdfLink)&(dffull.index<x)].id.count()))
  if dffull[(dffull.rpdPdfLink==rpdPdfLink)&(dffull.index<x)].id.count()==0:
   new=True
   tables = tabula.read_pdf(rpdPdfLink, pages='all', pandas_options={'header': None})
  if new==False:
   print(dffull[(dffull.rpdPdfLink==rpdPdfLink)&(dffull.index<x)]['num_d'].to_list()[0])
   dffull.loc[x,'num_d']=0 #dffull[(dffull.rpdPdfLink==rpdPdfLink)&(dffull.index<x)]['num_d'].to_list()[0] 
   dffull.loc[x,'num_r']=0 #dffull[(dffull.rpdPdfLink==rpdPdfLink)&(dffull.index<x)]['num_r'].to_list()[0]
   dffull.loc[x,'num_m']=0 #dffull[(dffull.rpdPdfLink==rpdPdfLink)&(dffull.index<x)]['num_m'].to_list()[0]
  print(str(x)+' :: '+str(dffull.iloc[x]['id']))
  i=0
  j=0
  k=0
  str_d=''
  str_m=''
  str_r=''
  num_d=0
  num_m=0
  num_r=0
  while (k<len(tables))&(new==True):
 # print('True!')
   i=0
   while i<len(tables[k].axes[1]):
    j=0
    while j<len(tables[k]):
     ffindr=0
     ffindm=0
     ffindd=0
     if str(tables[k][i][j])!='nan' and  str(type(tables[k][i][j]))!="<class 'numpy.float64'>" and  str(type(tables[k][i][j]))!="<class 'numpy.int64'>" :
      if ((str(tables[k][i][j].replace('\r', '')).find('Количество договоров, заключенных с использованием счетов эскроу:')>-1)or(str(tables[k][i][j].replace('\r', '')).find('Количество договоров, заключенных при условии уплаты обязательных')>-1)or(str(tables[k][i][j].replace('\r', '')).find('Количество договоров, заключенных с использованием иных')>-1))&(str(tables[k][i][j].replace('\r', '')).find('19.7.1.1.1')>-1)&(str(tables[k][i][j].replace('\r', '')).find('Жилые помещения')==-1):
  #      print(str(k)+'m|m'+str(i)+'|'+str(j)+'|'+str(tables[k][i][j]))
         str_d=str(str(tables[k][i][j]).replace('\r', '').replace('19.7.1.1.1.1','').replace('19.7.1.1.1.2','').replace('19.7.1.1.1.3',''))
         str_d=str_d.replace('Количество договоров, заключенных с использованием счетов эскроу:', '')
         str_d=str_d.replace('Количество договоров, заключенных с использованием счетов эскроу:', '')
         str_d=str_d.replace('Количество договоров, заключенных при условии уплаты обязательных отчислений(взносов) в компенсационный фонд:', '')
         str_d=str_d.replace('Количество договоров, заключенных при условии уплаты обязательных отчислений (взносов) в компенсационный фонд:', '')
         str_d=str_d.replace('Количество договоров, заключенных с использованием иных способов обеспечения гражданской ответственности застройщика:', '')
         str_d=str_d.replace('Количество договоров, заключенных с использованием иных способов обеспечениягражданской ответственности застройщика:', '')
         str_d=str_d.replace('(','').replace(')','').replace(' ','').replace(',','.').replace('Жилыепомещения:','')
         print(str(k)+'+'+str(i)+'+'+str(j)+'||'+str(len(str_d))+'^^^'+str_d)
         if len(str_d)>0:
          if math.isnan(float(str_d))==False: 
           num_d=num_d+float(str_d)
           ffindd=1
         if ffindd==0:
          str_d=str(str(tables[k][i][j+1]).replace('\r', '').replace('19.7.1.1.1.1','').replace('19.7.1.1.1.2','').replace('19.7.1.1.1.3',''))
          str_d=str_d.replace('Количество договоров, заключенных с использованием счетов эскроу:', '')
          str_d=str_d.replace('Количество договоров, заключенных с использованием счетов эскроу:', '')
          str_d=str_d.replace('Количество договоров, заключенных при условии уплаты обязательных отчислений(взносов) в компенсационный фонд:', '')
          str_d=str_d.replace('Количество договоров, заключенных при условии уплаты обязательных отчислений (взносов) в компенсационный фонд:', '')
          str_d=str_d.replace('Количество договоров, заключенных с использованием иных способов обеспечения гражданской ответственности застройщика:', '')
          str_d=str_d.replace('Количество договоров, заключенных с использованием иных способов обеспечениягражданской ответственности застройщика:', '')
          str_d=str_d.replace('(','').replace(')','').replace(' ','').replace(',','.').replace('Жилыепомещения:','')
          if len(str_d)>0:
           if math.isnan(float(str_d))==False: 
            num_d=num_d+float(str_d)
            ffindd=1
         if ffindd==0:
          str_d=str(str(tables[k][i+1][j+1]).replace('\r', '').replace('19.7.1.1.1.1','').replace('19.7.1.1.1.2','').replace('19.7.1.1.1.3',''))
          str_d=str_d.replace('Количество договоров, заключенных с использованием счетов эскроу:', '')
          str_d=str_d.replace('Количество договоров, заключенных с использованием счетов эскроу:', '')
          str_d=str_d.replace('Количество договоров, заключенных при условии уплаты обязательных отчислений(взносов) в компенсационный фонд:', '')
          str_d=str_d.replace('Количество договоров, заключенных при условии уплаты обязательных отчислений (взносов) в компенсационный фонд:', '')
          str_d=str_d.replace('Количество договоров, заключенных с использованием иных способов обеспечения гражданской ответственности застройщика:', '')
          str_d=str_d.replace('Количество договоров, заключенных с использованием иных способов обеспечениягражданской ответственности застройщика:', '')
          str_d=str_d.replace('(','').replace(')','').replace(' ','').replace(',','.').replace('Жилыепомещения:','')
          if len(str_d)>0:
           if math.isnan(float(str_d))==False: 
            num_d=num_d+float(str_d)
            ffindd=1
      if (str(tables[k][i][j].replace('\r', '')).find('Количество договоров, заключенных ')==-1)&(str(tables[k][i][j].replace('\r', '')).find('19.7.1.1.1')>-1)&(str(tables[k][i][j].replace('\r', '')).find('Добавлены сведения в пункты:')==-1)&(str(tables[k][i][j].replace('\r', '')).find('Жилые помещения')==-1):
        try: 
         str_d=str(str(tables[k][i+1][j]).replace('\r', '').replace('19.7.1.1.1.1','').replace('19.7.1.1.1.2','').replace('19.7.1.1.1.3',''))
         str_d=str_d.replace('Количество договоров, заключенных с использованием счетов эскроу:', '')
         str_d=str_d.replace('Количество договоров, заключенных с использованием счетов эскроу:', '')
         str_d=str_d.replace('Количество договоров, заключенных при условии уплаты обязательных отчислений(взносов) в компенсационный фонд:', '')
         str_d=str_d.replace('Количество договоров, заключенных при условии уплаты обязательных отчислений (взносов) в компенсационный фонд:', '')
         str_d=str_d.replace('Количество договоров, заключенных с использованием иных способов обеспечения гражданской ответственности застройщика:', '')
         str_d=str_d.replace('Количество договоров, заключенных с использованием иных способов обеспечениягражданской ответственности застройщика:', '')
         str_d=str_d.replace('(','').replace(')','').replace(' ','').replace(',','.').replace('Жилыепомещения:','')
         if len(str_d)>0:
           if math.isnan(float(str_d))==False: 
    #        print('111')
            num_d=num_d+float(str_d)
            ffindd=1
         if ffindd==0:
    #      print('222') 
          str_d=str(str(tables[k][i+2][j]).replace('\r', '').replace('19.7.1.1.1.1','').replace('19.7.1.1.1.2','').replace('19.7.1.1.1.3',''))
          str_d=str_d.replace('Количество договоров, заключенных с использованием счетов эскроу:', '')
          str_d=str_d.replace('Количество договоров, заключенных с использованием счетов эскроу:', '')
          str_d=str_d.replace('Количество договоров, заключенных при условии уплаты обязательных отчислений(взносов) в компенсационный фонд:', '')
          str_d=str_d.replace('Количество договоров, заключенных при условии уплаты обязательных отчислений (взносов) в компенсационный фонд:', '')
          str_d=str_d.replace('Количество договоров, заключенных с использованием иных способов обеспечения гражданской ответственности застройщика:', '')
          str_d=str_d.replace('Количество договоров, заключенных с использованием иных способов обеспечениягражданской ответственности застройщика:', '')
          str_d=str_d.replace('(','').replace(')','').replace(' ','').replace(',','.').replace('Жилыепомещения:','')
          if len(str_d)>0:
           if math.isnan(float(str_d))==False: 
            num_d=num_d+float(str_d)
            ffindd=1
         if ffindd==0:
    #      print('333')
          str_d=str(str(tables[k][i+1][j+1]).replace('\r', '').replace('19.7.1.1.1.1','').replace('19.7.1.1.1.2','').replace('19.7.1.1.1.3',''))
          str_d=str_d.replace('Количество договоров, заключенных с использованием счетов эскроу:', '')
          str_d=str_d.replace('Количество договоров, заключенных с использованием счетов эскроу:', '')
          str_d=str_d.replace('Количество договоров, заключенных при условии уплаты обязательных отчислений(взносов) в компенсационный фонд:', '')
          str_d=str_d.replace('Количество договоров, заключенных при условии уплаты обязательных отчислений (взносов) в компенсационный фонд:', '')
          str_d=str_d.replace('Количество договоров, заключенных с использованием иных способов обеспечения гражданской ответственности застройщика:', '')
          str_d=str_d.replace('Количество договоров, заключенных с использованием иных способов обеспечениягражданской ответственности застройщика:', '')
          str_d=str_d.replace('(','').replace(')','').replace(' ','').replace(',','.').replace('Жилыепомещения:','')
          if len(str_d)>0:
           if math.isnan(float(str_d))==False: 
            num_d=num_d+float(str_d)
            ffindd=1
        except (KeyError, ValueError):
   #      print('444') 
         try:
          str_d=str(str(tables[k][i+2][j]).replace('\r', '').replace('19.7.1.1.1.1','').replace('19.7.1.1.1.2','').replace('19.7.1.1.1.3',''))
          str_d=str_d.replace('Количество договоров, заключенных с использованием счетов эскроу:', '')
          str_d=str_d.replace('Количество договоров, заключенных с использованием счетов эскроу:', '')
          str_d=str_d.replace('Количество договоров, заключенных при условии уплаты обязательных отчислений(взносов) в компенсационный фонд:', '')
          str_d=str_d.replace('Количество договоров, заключенных при условии уплаты обязательных отчислений (взносов) в компенсационный фонд:', '')
          str_d=str_d.replace('Количество договоров, заключенных с использованием иных способов обеспечения гражданской ответственности застройщика:', '')
          str_d=str_d.replace('Количество договоров, заключенных с использованием иных способов обеспечениягражданской ответственности застройщика:', '')
          str_d=str_d.replace('(','').replace(')','').replace(' ','').replace(',','.').replace('Жилыепомещения:','')
     #     print('try')
          if len(str_d)>0:
           if math.isnan(float(str_d))==False: 
            num_d=num_d+float(str_d)
            ffindd=1
         except (KeyError, ValueError):
          try:
           str_d=str(str(tables[k][i+1][j+1]).replace('\r', '').replace('19.7.1.1.1.1','').replace('19.7.1.1.1.2','').replace('19.7.1.1.1.3',''))
           str_d=str_d.replace('Количество договоров, заключенных с использованием счетов эскроу:', '')
           str_d=str_d.replace('Количество договоров, заключенных с использованием счетов эскроу:', '')
           str_d=str_d.replace('Количество договоров, заключенных при условии уплаты обязательных отчислений(взносов) в компенсационный фонд:', '')
           str_d=str_d.replace('Количество договоров, заключенных при условии уплаты обязательных отчислений (взносов) в компенсационный фонд:', '')
           str_d=str_d.replace('Количество договоров, заключенных с использованием иных способов обеспечения гражданской ответственности застройщика:', '')
           str_d=str_d.replace('Количество договоров, заключенных с использованием иных способов обеспечениягражданской ответственности застройщика:', '')
           str_d=str_d.replace('(','').replace(')','').replace(' ','').replace(',','.').replace('Жилыепомещения:','')
           if len(str_d)>0:
            if math.isnan(float(str_d))==False: 
             num_d=num_d+float(str_d)
             ffindd=1 
          except (KeyError, ValueError):   
           pass
# ПЛОЩАДЬ
      if ((str(tables[k][i][j].replace('\r', '')).find('Площадь объектов, в отношении которых договор участия в долевом строительстве')>-1))&(str(tables[k][i][j].replace('\r', '')).find('19.7.2.1.1.')>-1)&(str(tables[k][i][j].replace('\r', '')).find('Добавлены сведения в пункты:')==-1)&(str(tables[k][i][j].replace('\r', '')).find('Жилые помещения')==-1):
 #       print(str(k)+'m|m'+str(i)+'|'+str(j)+'|'+str(tables[k][i][j]))
         str_m=str(str(tables[k][i][j]).replace('\r', '').replace('19.7.2.1.1.1','').replace('19.7.2.1.1.2','').replace('19.7.2.1.1.3','').replace('м2',''))
         str_m=str_m.replace('Площадь объектов, в отношении которых договор участия в долевом строительствезаключен с использованием счетов эскроу:', '')
         str_m=str_m.replace('Площадь объектов, в отношении которых договор участия в долевом строительстве заключен с использованием счетов эскроу:', '')
         str_m=str_m.replace('Площадь объектов, в отношении которых договор участия в долевом строительстве заключен с использованием иных способов обеспечения гражданскойответственности застройщика:', '')
         str_m=str_m.replace('Площадь объектов, в отношении которых договор участия в долевом строительстве заключен при условии уплаты обязательных отчислений (взносов) вкомпенсационный фонд:', '')
         str_m=str_m.replace('Площадь объектов, в отношении которых договор участия в долевом строительствезаключен при условии уплаты обязательных отчислений (взносов) в компенсационныйфонд:', '')
         str_m=str_m.replace('Площадь объектов, в отношении которых договор участия в долевом строительствезаключен с использованием иных способов обеспечения гражданской ответственностизастройщика:', '')
         str_m=str_m.replace('(','').replace(')','').replace(' ','').replace(',','.').replace('Жилыепомещения:','')
         if len(str_m)>0:
           if math.isnan(float(str_m))==False: 
            num_m=num_m+float(str_m)
            ffindm=1
         if ffindm==0:
          str_m=str(str(tables[k][i][j+1]).replace('\r', '').replace('19.7.2.1.1.1','').replace('19.7.2.1.1.2','').replace('19.7.2.1.1.3','').replace('м2',''))
          str_m=str_m.replace('Площадь объектов, в отношении которых договор участия в долевом строительствезаключен с использованием счетов эскроу:', '')
          str_m=str_m.replace('Площадь объектов, в отношении которых договор участия в долевом строительстве заключен с использованием счетов эскроу:', '')
          str_m=str_m.replace('Площадь объектов, в отношении которых договор участия в долевом строительстве заключен с использованием иных способов обеспечения гражданскойответственности застройщика:', '')
          str_m=str_m.replace('Площадь объектов, в отношении которых договор участия в долевом строительстве заключен при условии уплаты обязательных отчислений (взносов) вкомпенсационный фонд:', '')
          str_m=str_m.replace('Площадь объектов, в отношении которых договор участия в долевом строительствезаключен при условии уплаты обязательных отчислений (взносов) в компенсационныйфонд:', '')
          str_m=str_m.replace('Площадь объектов, в отношении которых договор участия в долевом строительствезаключен с использованием иных способов обеспечения гражданской ответственностизастройщика:', '')
          str_m=str_m.replace('(','').replace(')','').replace(' ','').replace(',','.').replace('Жилыепомещения:','')
          if len(str_m)>0:
           if math.isnan(float(str_m))==False: 
            num_m=num_m+float(str_m)
            ffindm=1
         if ffindm==0:
          str_m=str(str(tables[k][i+1][j+1]).replace('\r', '').replace('19.7.2.1.1.1','').replace('19.7.2.1.1.2','').replace('19.7.2.1.1.3','').replace('м2',''))
          str_m=str_m.replace('Площадь объектов, в отношении которых договор участия в долевом строительствезаключен с использованием счетов эскроу:', '')
          str_m=str_m.replace('Площадь объектов, в отношении которых договор участия в долевом строительстве заключен с использованием счетов эскроу:', '')
          str_m=str_m.replace('Площадь объектов, в отношении которых договор участия в долевом строительстве заключен с использованием иных способов обеспечения гражданскойответственности застройщика:', '')
          str_m=str_m.replace('Площадь объектов, в отношении которых договор участия в долевом строительстве заключен при условии уплаты обязательных отчислений (взносов) вкомпенсационный фонд:', '')
          str_m=str_m.replace('Площадь объектов, в отношении которых договор участия в долевом строительствезаключен при условии уплаты обязательных отчислений (взносов) в компенсационныйфонд:', '')
          str_m=str_m.replace('Площадь объектов, в отношении которых договор участия в долевом строительствезаключен с использованием иных способов обеспечения гражданской ответственностизастройщика:', '')
          str_m=str_m.replace('(','').replace(')','').replace(' ','').replace(',','.').replace('Жилыепомещения:','')
          if len(str_m)>0:
           if math.isnan(float(str_m))==False: 
            num_m=num_m+float(str_m)
            ffindm=1
      if (str(tables[k][i][j].replace('\r', '')).find('Площадь объектов, в отношении которых договор участия в долевом строительстве')==-1)&(str(tables[k][i][j].replace('\r', '')).find('19.7.2.1.1.')>-1)&(str(tables[k][i][j].replace('\r', '')).find('Добавлены сведения в пункты:')==-1)&(str(tables[k][i][j].replace('\r', '')).find('Жилые помещения')==-1):
        try: 
         str_m=str(str(tables[k][i+1][j]).replace('\r', '').replace('19.7.2.1.1.1','').replace('19.7.2.1.1.2','').replace('19.7.2.1.1.3','').replace('м2',''))
         str_m=str_m.replace('Площадь объектов, в отношении которых договор участия в долевом строительствезаключен с использованием счетов эскроу:', '')
         str_m=str_m.replace('Площадь объектов, в отношении которых договор участия в долевом строительстве заключен с использованием счетов эскроу:', '')
         str_m=str_m.replace('Площадь объектов, в отношении которых договор участия в долевом строительстве заключен с использованием иных способов обеспечения гражданскойответственности застройщика:', '')
         str_m=str_m.replace('Площадь объектов, в отношении которых договор участия в долевом строительстве заключен при условии уплаты обязательных отчислений (взносов) вкомпенсационный фонд:', '')
         str_m=str_m.replace('Площадь объектов, в отношении которых договор участия в долевом строительствезаключен при условии уплаты обязательных отчислений (взносов) в компенсационныйфонд:', '')
         str_m=str_m.replace('Площадь объектов, в отношении которых договор участия в долевом строительствезаключен с использованием иных способов обеспечения гражданской ответственностизастройщика:', '')
         str_m=str_m.replace('(','').replace(')','').replace(' ','').replace(',','.').replace('Жилыепомещения:','')
         if len(str_m)>0:
           if math.isnan(float(str_m))==False: 
            num_m=num_m+float(str_m)
            ffindm=1
         if ffindm==0:
          str_m=str(str(tables[k][i+2][j]).replace('\r', '').replace('19.7.2.1.1.1','').replace('19.7.2.1.1.2','').replace('19.7.2.1.1.3','').replace('м2',''))
          str_m=str_m.replace('Площадь объектов, в отношении которых договор участия в долевом строительствезаключен с использованием счетов эскроу:', '')
          str_m=str_m.replace('Площадь объектов, в отношении которых договор участия в долевом строительстве заключен с использованием счетов эскроу:', '')
          str_m=str_m.replace('Площадь объектов, в отношении которых договор участия в долевом строительстве заключен с использованием иных способов обеспечения гражданскойответственности застройщика:', '')
          str_m=str_m.replace('Площадь объектов, в отношении которых договор участия в долевом строительстве заключен при условии уплаты обязательных отчислений (взносов) вкомпенсационный фонд:', '')
          str_m=str_m.replace('Площадь объектов, в отношении которых договор участия в долевом строительствезаключен при условии уплаты обязательных отчислений (взносов) в компенсационныйфонд:', '')
          str_m=str_m.replace('Площадь объектов, в отношении которых договор участия в долевом строительствезаключен с использованием иных способов обеспечения гражданской ответственностизастройщика:', '')
          str_m=str_m.replace('(','').replace(')','').replace(' ','').replace(',','.').replace('Жилыепомещения:','')
          if len(str_m)>0:
           if math.isnan(float(str_m))==False: 
            num_m=num_m+float(str_m)
            ffindm=1
        except (KeyError, ValueError):
         try:
          str_m=str(str(tables[k][i+2][j]).replace('\r', '').replace('19.7.2.1.1.1','').replace('19.7.2.1.1.2','').replace('19.7.2.1.1.3','').replace('м2',''))
          str_m=str_m.replace('Площадь объектов, в отношении которых договор участия в долевом строительствезаключен с использованием счетов эскроу:', '')
          str_m=str_m.replace('Площадь объектов, в отношении которых договор участия в долевом строительстве заключен с использованием счетов эскроу:', '')
          str_m=str_m.replace('Площадь объектов, в отношении которых договор участия в долевом строительстве заключен с использованием иных способов обеспечения гражданскойответственности застройщика:', '')
          str_m=str_m.replace('Площадь объектов, в отношении которых договор участия в долевом строительстве заключен при условии уплаты обязательных отчислений (взносов) вкомпенсационный фонд:', '')
          str_m=str_m.replace('Площадь объектов, в отношении которых договор участия в долевом строительствезаключен при условии уплаты обязательных отчислений (взносов) в компенсационныйфонд:', '')
          str_m=str_m.replace('Площадь объектов, в отношении которых договор участия в долевом строительствезаключен с использованием иных способов обеспечения гражданской ответственностизастройщика:', '')
          str_m=str_m.replace('(','').replace(')','').replace(' ','').replace(',','.').replace('Жилыепомещения:','')
          if len(str_m)>0:
           if math.isnan(float(str_m))==False: 
            num_m=num_m+float(str_m)
            ffindm=1
         except (KeyError, ValueError):
           pass
# РУБЛИ
      ffindr=0
      if ((str(tables[k][i][j].replace('\r', '')).find('Суммарная цена договоров, заключенных с использованием иных способов обеспечениягражданской ответственности застройщика')>-1)or(str(tables[k][i][j].replace('\r', '')).find('Суммарная цена договоров, заключенных при условии уплаты обязательных отчислений(взносов) в компенсационный фонд')>-1)or(str(tables[k][i][j].replace('\r', '')).find('Суммарная цена договоров, заключенных с использованием счетов эскроу')>-1))&(str(tables[k][i][j].replace('\r', '')).find('19.7.3.1.1.')>-1)&(str(tables[k][i][j].replace('\r', '')).find('Добавлены сведения в пункты:')==-1)&(str(tables[k][i][j].replace('\r', '')).find('Жилые помещения')==-1):
         print(str(k)+'m|m'+str(i)+'|'+str(j)+'|'+str(tables[k][i][j]))
         str_r=str(str(tables[k][i][j]).replace('\r', '').replace('19.7.3.1.1.1','').replace('19.7.3.1.1.2','').replace('19.7.3.1.1.3','').replace('руб.','').replace('руб',''))
         str_r=str_r.replace('Суммарная цена договоров, заключенных с использованием счетов эскроу:', '')
         str_r=str_r.replace('Суммарная цена договоров, заключенных при условии уплаты обязательных отчислений(взносов) в компенсационный фонд:', '')
         str_r=str_r.replace('Суммарная цена договоров, заключенных с использованием иных способов обеспечениягражданской ответственности застройщика:', '')
         str_r=str_r.replace('(','').replace(')','').replace(' ','').replace(',','.').replace('Жилыепомещения:','')
         if len(str_r)>0:
           if math.isnan(float(str_r))==False: 
            num_r=num_r+float(str_r)
            ffindr=1
         if ffindr==0:
          str_r=str(str(tables[k][i][j+1]).replace('\r', '').replace('19.7.3.1.1.1','').replace('19.7.3.1.1.2','').replace('19.7.3.1.1.3','').replace('руб.','').replace('руб',''))
          str_r=str_r.replace('Суммарная цена договоров, заключенных с использованием счетов эскроу:', '')
          str_r=str_r.replace('Суммарная цена договоров, заключенных при условии уплаты обязательных отчислений(взносов) в компенсационный фонд:', '')
          str_r=str_r.replace('Суммарная цена договоров, заключенных с использованием иных способов обеспечениягражданской ответственности застройщика:', '')
          str_r=str_r.replace('(','').replace(')','').replace(' ','').replace(',','.').replace('Жилыепомещения:','')
          if len(str_r)>0:
           if math.isnan(float(str_r))==False: 
            num_r=num_r+float(str_r)
            ffindr=1
         if ffindr==0:
          str_r=str(str(tables[k][i+1][j+1]).replace('\r', '').replace('19.7.3.1.1.1','').replace('19.7.3.1.1.2','').replace('19.7.3.1.1.3','').replace('руб.','').replace('руб',''))
          str_r=str_r.replace('Суммарная цена договоров, заключенных с использованием счетов эскроу:', '')
          str_r=str_r.replace('Суммарная цена договоров, заключенных при условии уплаты обязательных отчислений(взносов) в компенсационный фонд:', '')
          str_r=str_r.replace('Суммарная цена договоров, заключенных с использованием иных способов обеспечениягражданской ответственности застройщика:', '')
          str_r=str_r.replace('(','').replace(')','').replace(' ','').replace(',','.').replace('Жилыепомещения:','')
          if len(str_r)>0:
           if math.isnan(float(str_r))==False: 
            num_r=num_r+float(str_r)
            ffindr=1
      if (str(tables[k][i][j].replace('\r', '')).find('Суммарная цена договоров, заключенных')==-1)&(str(tables[k][i][j].replace('\r', '')).find('19.7.3.1.1.')>-1)&(str(tables[k][i][j].replace('\r', '')).find('Добавлены сведения в пункты:')==-1)&(str(tables[k][i][j].replace('\r', '')).find('Жилые помещения')==-1):
        try: 
         str_r=str(str(tables[k][i+1][j]).replace('\r', '').replace('19.7.3.1.1.1','').replace('19.7.3.1.1.2','').replace('19.7.3.1.1.3','').replace('руб.','').replace('руб',''))
         str_r=str_r.replace('Суммарная цена договоров, заключенных с использованием счетов эскроу:', '')
         str_r=str_r.replace('Суммарная цена договоров, заключенных при условии уплаты обязательных отчислений(взносов) в компенсационный фонд:', '')
         str_r=str_r.replace('Суммарная цена договоров, заключенных с использованием иных способов обеспечениягражданской ответственности застройщика:', '')
         str_r=str_r.replace('(','').replace(')','').replace(' ','').replace(',','.').replace('Жилыепомещения:','')
         if len(str_r)>0:
           if math.isnan(float(str_r))==False: 
            num_r=num_r+float(str_r)
            ffindr=1
         if ffindr==0:
          str_r=str(str(tables[k][i+2][j]).replace('\r', '').replace('19.7.3.1.1.1','').replace('19.7.3.1.1.2','').replace('19.7.3.1.1.3','').replace('руб.','').replace('руб',''))
          str_r=str_r.replace('Суммарная цена договоров, заключенных с использованием счетов эскроу:', '')
          str_r=str_r.replace('Суммарная цена договоров, заключенных при условии уплаты обязательных отчислений(взносов) в компенсационный фонд:', '')
          str_r=str_r.replace('Суммарная цена договоров, заключенных с использованием иных способов обеспечениягражданской ответственности застройщика:', '')
          str_r=str_r.replace('(','').replace(')','').replace(' ','').replace(',','.').replace('Жилыепомещения:','')
          if len(str_r)>0:
           if math.isnan(float(str_r))==False: 
            num_r=num_r+float(str_r)
            ffindr=1
        except (KeyError, ValueError):
         try:
          str_r=str(str(tables[k][i+2][j]).replace('\r', '').replace('19.7.3.1.1.1','').replace('19.7.3.1.1.2','').replace('19.7.3.1.1.3','').replace('руб.','').replace('руб',''))
          str_r=str_r.replace('Суммарная цена договоров, заключенных с использованием счетов эскроу:', '')
          str_r=str_r.replace('Суммарная цена договоров, заключенных при условии уплаты обязательных отчислений(взносов) в компенсационный фонд:', '')
          str_r=str_r.replace('Суммарная цена договоров, заключенных с использованием иных способов обеспечениягражданской ответственности застройщика:', '')
          str_r=str_r.replace('(','').replace(')','').replace(' ','').replace(',','.').replace('Жилыепомещения:','')
          if len(str_r)>0:
           if math.isnan(float(str_r))==False: 
            num_r=num_r+float(str_r)
            ffindr=1
         except (KeyError, ValueError):
           pass
        if ffindr==0:
         try:
          str_r=str(str(tables[k][i][j+1]).replace('\r', '').replace('19.7.3.1.1.1','').replace('19.7.3.1.1.2','').replace('19.7.3.1.1.3','').replace('руб.','').replace('руб',''))
          str_r=str_r.replace('Суммарная цена договоров, заключенных с использованием счетов эскроу:', '')
          str_r=str_r.replace('Суммарная цена договоров, заключенных при условии уплаты обязательных отчислений(взносов) в компенсационный фонд:', '')
          str_r=str_r.replace('Суммарная цена договоров, заключенных с использованием иных способов обеспечениягражданской ответственности застройщика:', '')
          str_r=str_r.replace('(','').replace(')','').replace(' ','').replace(',','.').replace('Жилыепомещения:','')
          if len(str_r)>0:
           if math.isnan(float(str_r))==False: 
            num_r=num_r+float(str_r)
            ffindr=1
         except:
           pass
     #print(str(k)+'|'+str(i)+'|'+str(j)) 
     j=j+1
    i=i+1
#   print(str(k))
   k=k+1
  print(str(num_d)+'|'+str(num_m)+'|'+str(num_r))
  if new==True:
   dffull.loc[x,'num_d']=num_d
   dffull.loc[x,'num_r']=num_r
   dffull.loc[x,'num_m']=num_m
  x=x+1
#  print(dffull.objId[x])
#  print(num_r)
#  print(num_m)
# 42767, 43489, 40237, 40238,40236,40733,41623-ЮГ, 48688, 48686, 48687, 46470, 46469
# 40733


https://xn--80az8a.xn--d1aqf.xn--p1ai/api/ext/file/c7da75eb-96ca-4844-ab40-ded5494fc86a
0:0
0 :: 18136
65.0|9761.87|457831535.06
https://xn--80az8a.xn--d1aqf.xn--p1ai/api/ext/file/50d90d78-98fe-49fa-bcf9-92e1c376a655
1:0
1 :: 4024
444.0|35363.66|1840548100.0
https://xn--80az8a.xn--d1aqf.xn--p1ai/api/ext/file/50d90d78-98fe-49fa-bcf9-92e1c376a655
2:1
444.0
2 :: 4025
0|0|0
https://xn--80az8a.xn--d1aqf.xn--p1ai/api/ext/file/50d90d78-98fe-49fa-bcf9-92e1c376a655
3:2
444.0
3 :: 4023
0|0|0
https://xn--80az8a.xn--d1aqf.xn--p1ai/api/ext/file/50d90d78-98fe-49fa-bcf9-92e1c376a655
4:3
444.0
4 :: 4026
0|0|0
https://xn--80az8a.xn--d1aqf.xn--p1ai/api/ext/file/10ec2232-68bc-4a71-86b3-d587db8280ca
5:0
5 :: 47153
3.0|1765.2600000000002|89136803.7
https://xn--80az8a.xn--d1aqf.xn--p1ai/api/ext/file/10ec2232-68bc-4a71-86b3-d587db8280ca
6:1
3.0
6 :: 47152
0|0|0
https://xn--80az8a.xn--d1aqf.xn--p1ai/api/ext/file/b347e8d4-20da-43f1-be0a-016299909816
7:0
7 :: 24539
170.0|16989.82|870762368.0
https://xn--80az8a.

In [ ]:
data_table.DataTable.max_columns = 110
data_table.DataTable(dffull, include_index=False, num_rows_per_page=200)

,id,hobjId,pdId,region,address,floorMin,floorMax,objElemLivingCnt,objReady100PercDt,wallMaterialShortDesc,...,metro.color,metro.time,metro.isWalk,metro.colors,nameObj,complexShortNm,raspr,num_d,num_r,num_m
0,18136,20249,43084,63,"г Самара, улица Советской Армии, д. 0, корпус ...",27,27,360,2023-06-30,Монолит-кирпич,...,#CD0505,11.500000,True,[#CD0505],NaN,NaN,176.0,65.0,4.578315e+08,9761.87
1,4024,23666,2200,63,"г Самара, район Октябрьский, просека Пятая",5,5,16,2023-12-31,Кирпич,...,NaN,NaN,NaN,NaN,"Жилой комплекс ""Акварель"" (ЖК ""Акварель"") дом 9",Акварель,3.0,444.0,1.840548e+09,35363.66
2,4025,23664,2200,63,"г Самара, район Октябрьский, просека Пятая",5,5,64,2022-12-31,Кирпич,...,NaN,NaN,NaN,NaN,"Жилой комплекс ""Акварель"" (ЖК ""Акварель"") дом 7",Акварель,64.0,0.0,0.000000e+00,0.00
3,4023,23665,2200,63,"г Самара, район Октябрьский, просека Пятая",5,5,48,2022-12-31,Кирпич,...,NaN,NaN,NaN,NaN,"Жилой комплекс ""Акварель"" (ЖК ""Акварель"") дом 8",Акварель,48.0,0.0,0.000000e+00,0.00
4,4026,23667,2200,63,"г Самара, район Октябрьский, просека Пятая",5,5,64,2022-12-31,Кирпич,...,NaN,NaN,NaN,NaN,"Жилой комплекс ""Акварель"" (ЖК ""Акварель"") дом 10",Акварель,64.0,0.0,0.000000e+00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182,49394,130035,516724,63,"г Тольятти, район Автозаводский, ул 40 лет Поб...",8,9,47,2023-12-31,Другое,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.000000e+00,0.00
183,49391,130029,542409,63,"г Самара, шоссе Московское, блок-секция 2.1, 2...",17,17,602,2023-12-31,Другое,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.000000e+00,0.00
184,49396,130033,516724,63,"г Тольятти, район Автозаводский, ул 40 лет Поб...",14,17,117,2023-12-31,Другое,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.000000e+00,0.00
185,34040,44994,232480,63,"г Самара, ул Советской Армии, д. 127",26,26,496,2023-06-30,Монолит-кирпич,...,#CD0505,10.150000,False,[#CD0505],"ЖК ""Журавли""",NaN,496.0,496.0,1.201525e+09,22387.34
